In [1]:
import pandas as pd
import re
# coding: utf-8

# 만든 기능 임포트
from crawling import UrlCrawling, ContentCrawling
from preprocessing import Preprocessing
from clustering import Clustering
from summary import Summary
from remove import Remove
from database import insert, selectAll, selectToDay
from tts import tts

In [2]:
# url 크롤링
url_crawler = UrlCrawling()     # 기사 링크 크롤링 객체

six_url_df = url_crawler.getSixUrl()                            # 6개 카테고리 (카테고리, url 데이터프레임)
entertainment_url_df = url_crawler.getEntertainmentUrl()        # 연예 (카테고리, url 데이터프레임)
sports_url_df = url_crawler.getSportsUrl()                      # 스포츠 (카테고리, url 데이터프레임)

Remove.duplication(six_url_df, entertainment_url_df, sports_url_df)     # 이미 요약한 기사 제거

# 본문 크롤링
six_url, entertainment_url, sports_url = list(six_url_df['six_url']), list(entertainment_url_df['entertainment_url']), list(sports_url_df['sports_url'])
all_url = six_url + entertainment_url + sports_url      # 전체 url
category = list(six_url_df['category']) + list(entertainment_url_df['category']) + list(sports_url_df['category'])

content_crawler = ContentCrawling()     # 기사 크롤링 객체

content_crawler.getSixContent(six_url)                                          # 6개 카테고리 기사 크롤링
content_crawler.getEntertainmentContent(entertainment_url)                      # 연예 기사 크롤링
content_crawler.getSportsContent(sports_url)                                    # 스포츠 기사 크롤링

news_df = content_crawler.makeDataFrame(all_url, category)                      # 본문 데이터프레임 생성

Remove.photoNews(news_df)                                                       # 포토 기사 삭제
Remove.shortNews(news_df)                                                       # 3문장 이하 기사 삭제

news_df['content'] = news_df['content'].apply(lambda x : Preprocessing.clean(x)) # 본문 전처리

Preprocessing.getNouns(news_df)                                                 # 명사 추출

Remove.englishNews(news_df)                                                     # 영어 기사 삭제

vector_list = Preprocessing.getVector(news_df)                                  # 명사 벡터화

Clustering.addClusterNumber(news_df, vector_list)                               # 군집 번호 열 생성
cluster_counts_df = Clustering.getClusteredArticle(news_df)                     # 군집 개수 카운트한 df

summary_news = Summary.getSummaryArticle(news_df, cluster_counts_df)            # 요약한 기사 데이터 프레임 반환

Preprocessing.convertCategory(summary_news)                                     # 카테고리 이름을 번호로 변경

Remove.similarNews(summary_news)                                                # 오늘 요약한 기사와 유사한 기사 drop                       

정치 1 페이지
연예 1 페이지
스포츠 1 페이지
1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 
1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 
1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 


AssertionError: phrase input should be string, not <class 'list'>

In [ ]:
# insert(summary_news)

In [ ]:
# 필요할 때 쓰는 코드
# csv로 저장
# summary_news.to_csv("csv/유사도측정.csv",index=False, encoding="utf-8-sig")
# df = pd.read_csv("csv/유사도측정.csv")
# tts(df)
# insert(summary_news.values.tolist())

In [4]:
news_df['content'] = news_df['content'].apply(lambda x : Preprocessing.clean(x))                     # 본문 전처리
news_df

,category,title,content,img,url,summary
0,정치,"“이재명, 혈관재건술 후 회복 중…내경정맥 60% 손상”",더불어민주당은 2일 괴한으로부터 흉기 습격을 당한 이재명 대표에 대해...,https://imgnews.pstatic.net/image/366/2024/01/...,https://n.news.naver.com/mnews/article/366/000...,"더불어민주당은 2일 괴한으로부터 흉기 습격을 당한 이재명 대표에 대해 ""중증 수술을..."
1,정치,"장예찬 ""미안하지만 '제2 황교안'은 한동훈 아닌 이재명..국힘은 '으쌰으쌰' 분위...",유재광 앵커 서울광역방송센터입니다.\n 4·10 총선이 꼭 100일 앞으로 다가왔습...,,https://n.news.naver.com/mnews/article/660/000...,
2,정치,목소리 낮춘 여야 ‘자중 모드’…이낙연 탈당·창당 연기 가능성,이재명 더불어민주당 대표 피습 사건이 발생한 2일 여야가 일부 일정을 취소하며 차분...,https://imgnews.pstatic.net/image/032/2024/01/...,https://n.news.naver.com/mnews/article/032/000...,이재명 더불어민주당 대표 피습 사건이 발생한 2일 여야가 일부 일정을 취소하며 차분...
3,정치,"""이재명, 경정맥 손상…2시간 넘게 수술받고 회복 중""",부산을 방문했다가 60대 남성에게 목 부위를 피습당한 이재명 더불어민주당 대표가 2...,https://imgnews.pstatic.net/image/088/2024/01/...,https://n.news.naver.com/mnews/article/088/000...,부산을 방문했다가 60대 남성에게 목 부위를 피습당한 이재명 더불어민주당 대표가 2...
4,정치,"이 대표, 2시간 수술 후 중환자실로…“경동맥 안 건드려 다행”",부산 방문 중 흉기 습격을 당한 이재명 더불어민주당 대표는 2일 오후 서울 종로구 ...,https://imgnews.pstatic.net/image/032/2024/01/...,https://n.news.naver.com/mnews/article/032/000...,부산 방문 중 흉기 습격을 당한 이재명 더불어민주당 대표는 2일 오후 서울 종로구 ...
5,정치,예측불허 22대 총선.. 격동의 2024 전북 정치,앵커제22대 총선이 석달 앞으로 다가오면서 전북의 정치에도 큰 변화가 예상되고 있...,https://imgnews.pstatic.net/image/659/2024/01/...,https://n.news.naver.com/mnews/article/659/000...,제22대 총선이 석달 앞으로 다가오면서 전북의 정치에도 큰 변화가 예상되고 있습니다...
6,정치,"하태경 “조국, 이선균 수사에 음모론…그러니 국민이 운동권 특권정치 청산 요구”",하태경 국민의힘 의원은 배우 이선균의 마약 혐의 수사가 ‘국면 전환용’일 수 있다는...,https://imgnews.pstatic.net/image/021/2024/01/...,https://n.news.naver.com/mnews/article/021/000...,하태경 국민의힘 의원은 배우 이선균의 마약 혐의 수사가 '국면 전환용'일 수 있다는...
7,정치,"""이재명, 2시간 넘게 혈관 재건술…아직 퇴원 시점 예상 못해""",부산 방문 일정 도중 60대 남성에게 목 부위를 피습당한 이...,https://imgnews.pstatic.net/image/025/2024/01/...,https://n.news.naver.com/mnews/article/025/000...,
8,정치,총선 D-99 시점 야당 대표 흉기 피습…사건 파장은?,"앵커이 내용은 원종진 기자와 더 살펴보겠습니다.\n원종진 기자 네, 그렇습니다.\...",,https://n.news.naver.com/mnews/article/055/000...,
9,정치,"전문가들 “이재명 피습 부위, 경동맥이었다면 지혈도 어려워”",“경정맥 손상이라 서울 이송도 가능”이재명 더불어민주당 대표가 2일 부산 가덕도 신...,https://imgnews.pstatic.net/image/022/2024/01/...,https://n.news.naver.com/mnews/article/022/000...,이재명 더불어민주당 대표가 2일 부산 가덕도 신공항 부지 현장 방문 일정을 소화하던...
